<a href="https://colab.research.google.com/github/g4aidl-upc-winter-2020/3D-Shape-classification/blob/main/PointNet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all needed packages from PyG:
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
!pip install -q torch-geometric

     |████████████████████████████████| 2.6MB 5.0MB/s 
     |████████████████████████████████| 1.5MB 3.6MB/s 
     |████████████████████████████████| 194kB 5.7MB/s 
     |████████████████████████████████| 235kB 6.9MB/s 
     |████████████████████████████████| 2.2MB 7.2MB/s 
     |████████████████████████████████| 51kB 6.0MB/s 


In [2]:
import torch
from torch_geometric.datasets import ModelNet
from torch_geometric.data import DataLoader
from torch_geometric.utils import to_dense_batch
import torch_geometric.transforms as T
from torch_geometric.transforms import SamplePoints, NormalizeScale

import matplotlib.pyplot as plt

import sys

from sklearn.metrics import confusion_matrix

## Set a fixed seed

In [3]:
seed = 42

#Controlling sources of randomness
torch.manual_seed(seed)  #Sets the seed for generating random numbers for all devices (both CPU and CUDA)

#CUDA convolution benchmarking
torch.backends.cudnn.benchmark = False #ensures that CUDA selects the same algorithm each time an application is run

#Avoiding nondeterministic algorithms
torch.use_deterministic_algorithms(True) #use “deterministic” algorithms (given the same input always produce the same output)

### Import drive folder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# PointNet

## Dataset

In [5]:
# Import ModelNet10 dataset from PyG
test_dataset = ModelNet(root='/content/drive/MyDrive/Proyecto/Colabs/ModelNet', name="10", train=False, pre_transform=T.SamplePoints(num=1024))  #test dataset

In [6]:
print('Dataset info:')
print('--------------')
print('Test dataset size: ', len(test_dataset))

Dataset info:
--------------
Test dataset size:  908


### Normalize Input points

In [7]:
#Centers and normalizes node positions to the interval (-1,1) 
test_dataset.transform = NormalizeScale()

## Testing

### Make sure your runtime has a GPU

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
assert not device.type == 'cpu', "Change Runtime Type -> GPU"

### Loading the model architecture

In [9]:
#We include a new file path that will point to modules the we want to import
sys.path.append('/content/drive/MyDrive/Proyecto/Colabs/architectures/PointNet')  

In [10]:
#Import the model from a python script
from PointNet_Architecture import ClassificationPointNet
model = ClassificationPointNet()  #Instantiate the model
model.to(device)    # Pass the model to GPU(device)

ClassificationPointNet(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(64, 64, k

### Loading best parameters from training 

In [11]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Proyecto/Colabs/experiments/logs/PointNet/train/best_params.pt')) 

<All keys matched successfully>

### Dataloader

In [12]:
test_loader = DataLoader(test_dataset, batch_size=32)

### Test

In [13]:
model.eval()
with torch.no_grad():
  total=correct=0
  all_preds = []
  all_labels = []
  for data in test_loader:
      inputs = to_dense_batch(data.pos, batch=data.batch)  #return a tuple where in the first position there are the points of every pointcloud in the batch 

      output = model(inputs[0].to(device).float().transpose(1,2))  #Pass the inputs through the network and compute the outputs
      _, preds = torch.max(output, 1) # We get the maximum prediction value (correct category) for each pointcloud in the batch
      total += data.y.size(0)  # total number of samples in the test_loader
      correct += (preds == data.y.to(device)).sum().item()  #number of total correct predictions in the test_loader
      
      all_preds += list(preds.cpu().numpy())
      all_labels += list(data.y.cpu().numpy())

val_acc = 100. * (correct / total)
print('Valid accuracy: {:.2f} %'.format(val_acc))

Valid accuracy: 90.97 %


## Metrics

###Confusion Matrix

In [14]:
#ClassesNames = ['Bathtub', 'Bed', 'Chair', 'Desk', 'Dresser', 'Monitor', 'Night_Stand', 'Sofa', 'Table', 'Toilet']
#Classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [15]:
cm = confusion_matrix(all_labels, all_preds);
cm

array([[ 47,   1,   0,   0,   0,   0,   0,   0,   1,   1],
       [  0,  98,   0,   0,   0,   0,   0,   0,   2,   0],
       [  0,   0, 100,   0,   0,   0,   0,   0,   0,   0],
       [  0,   1,   0,  73,   1,   0,   1,   5,   5,   0],
       [  0,   0,   0,   1,  75,   0,   9,   0,   1,   0],
       [  0,   0,   0,   0,   0,  99,   1,   0,   0,   0],
       [  0,   1,   0,   1,  14,   0,  64,   0,   6,   0],
       [  0,   0,   0,   1,   0,   0,   1,  98,   0,   0],
       [  0,   0,   0,  26,   0,   0,   0,   0,  74,   0],
       [  0,   1,   1,   0,   0,   0,   0,   0,   0,  98]])

### Precision and Recall 

In [16]:
##Function to compute Precision and recall scores per class

dim = cm.shape[0]
ClassesNames = ['Bathtub', 'Bed', 'Chair', 'Desk', 'Dresser', 'Monitor', 'Night_Stand', 'Sofa', 'Table', 'Toilet']
# Precision = TP / (TP + FP)
# Recall = TP / (TP + FN)
Precision = []
Recall = []
Correct = 0
Samples = 0
PrecisionWAvg = 0
RecallWAvg = 0
for i in range(0, dim):
  TP = cm[i,i]  #Diagonal value (TP)
  FPc = 0
  FNc = 0
  for j in range(0, dim):
    Samples += cm[i,j]
    FNc += cm[i,j]  #Add all line values
    FPc += cm[j,i]  #Add all column values
  FN = FNc - TP   #Substract diagonal value (TP)
  FP = FPc - TP   #Substract diagonal value (TP)
  Correct += TP
  if TP==0:
    Precision.append(0)
    Recall.append(0)
  else:  
    Precision.append(100*(TP/(TP+FP)))
    Recall.append(100*(TP/(TP+FN)))
    PrecisionWAvg+=100*(TP/(TP+FP))*(TP+FN)
    RecallWAvg+=100*(TP/(TP+FN))*(TP+FN)

  print(ClassesNames[i], "\n\tPrecision: {:.2f}% \tRecall: {:.2f}%".format(Precision[i],Recall[i]))
  print("\tTP:", TP,"  FP:", FP,"  FN:", FN,"  Samples in Test:",TP+FN)

print("\nTOTAL Accuracy: {:.2f}%".format(100*Correct/Samples),"  Samples in Test: ", Samples,"  Correct Predictions: ", Correct)
print("Avg. Weighted Precision: {:.2f}% \tAvg. Weighted Recall: {:.2f}%\n".format(PrecisionWAvg/Samples, RecallWAvg/Samples))


Bathtub 
	Precision: 100.00% 	Recall: 94.00%
	TP: 47   FP: 0   FN: 3   Samples in Test: 50
Bed 
	Precision: 96.08% 	Recall: 98.00%
	TP: 98   FP: 4   FN: 2   Samples in Test: 100
Chair 
	Precision: 99.01% 	Recall: 100.00%
	TP: 100   FP: 1   FN: 0   Samples in Test: 100
Desk 
	Precision: 71.57% 	Recall: 84.88%
	TP: 73   FP: 29   FN: 13   Samples in Test: 86
Dresser 
	Precision: 83.33% 	Recall: 87.21%
	TP: 75   FP: 15   FN: 11   Samples in Test: 86
Monitor 
	Precision: 100.00% 	Recall: 99.00%
	TP: 99   FP: 0   FN: 1   Samples in Test: 100
Night_Stand 
	Precision: 84.21% 	Recall: 74.42%
	TP: 64   FP: 12   FN: 22   Samples in Test: 86
Sofa 
	Precision: 95.15% 	Recall: 98.00%
	TP: 98   FP: 5   FN: 2   Samples in Test: 100
Table 
	Precision: 83.15% 	Recall: 74.00%
	TP: 74   FP: 15   FN: 26   Samples in Test: 100
Toilet 
	Precision: 98.99% 	Recall: 98.00%
	TP: 98   FP: 1   FN: 2   Samples in Test: 100

TOTAL Accuracy: 90.97%   Samples in Test:  908   Correct Predictions:  826
Avg. Weighted Pre